In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
import torchtext
import tqdm

In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

from data.preprocessing import preprocess_dataset  # ✅ Import ulang setelah menambahkan path
from data.data_loader import get_data_loader
from utils.training import init_model, train_fn, evaluate_fn


# Import Dataset

In [4]:
!which python
!python --version


/opt/homebrew/anaconda3/envs/gru_translation/bin/python
Python 3.10.16


In [5]:
import datasets
print(datasets.__version__)

2.14.5


In [6]:
from datasets import load_from_disk
import os

dataset_path = os.path.abspath("../../data/indonesia_jawa_dataset")

dataset = load_from_disk(dataset_path)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['indonesia', 'jawa'],
        num_rows: 800
    })
    validation: Dataset({
        features: ['indonesia', 'jawa'],
        num_rows: 100
    })
    test: Dataset({
        features: ['indonesia', 'jawa'],
        num_rows: 100
    })
})


In [7]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

In [8]:
train_data[5]

{'indonesia': 'Mau bikin postingan yang isinya mengedukasi customers gojek.',
 'jawa': 'Pengin nggawe postingan sing isine ngajari pelanggan Gojek.'}

# Preprocessing

In [9]:
# Preprocessing
train_data, valid_data, test_data, en_vocab, id_vocab = preprocess_dataset(dataset)

Map: 100%|██████████| 100/100 [00:00<00:00, 461.43 examples/s]


✅ Tokenisasi selesai dengan BERT tokenizer!


Map: 100%|██████████| 100/100 [00:00<00:00, 1340.88 examples/s]


✅ Data siap digunakan dalam format PyTorch!


# Data Loader

In [10]:
# Ambil indeks padding dari vocabulary
pad_index = en_vocab["<pad>"]

# Definisikan batch size
batch_size = 128  # Sesuai kebutuhan

# Buat DataLoader untuk train, valid, dan test
train_loader = get_data_loader(train_data, batch_size=batch_size, pad_index=pad_index, shuffle=True)
valid_loader = get_data_loader(valid_data, batch_size=batch_size, pad_index=pad_index, shuffle=False)
test_loader = get_data_loader(test_data, batch_size=batch_size, pad_index=pad_index, shuffle=False)

# TRAIN

In [13]:
# Inisialisasi Model
input_dim = len(id_vocab)
output_dim = len(en_vocab)
embedding_dim = 512
hidden_dim = 1024
dropout = 0.3
clip = 1.0
teacher_forcing_initial = 0.5
teacher_forcing_final = 0.1
epochs = 10
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

model, optimizer, criterion = init_model(input_dim, output_dim, embedding_dim, hidden_dim, dropout, pad_index, device)
model

Using device: mps


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2622, 512)
    (rnn): GRU(512, 1024)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(3113, 512)
    (rnn): GRU(1536, 1024)
    (fc_out): Linear(in_features=2560, out_features=3113, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
)

In [ ]:
# Training Loop
best_valid_loss = float("inf")
patience = 5
patience_counter = 0

for epoch in tqdm.tqdm(range(epochs)):
    teacher_forcing_ratio = teacher_forcing_initial - \
                            (teacher_forcing_initial - teacher_forcing_final) * \
                            (epoch / (epochs - 1))
    
    train_loss = train_fn(model, train_loader, optimizer, criterion, clip, teacher_forcing_ratio, device)
    valid_loss = evaluate_fn(model, valid_loader, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "checkpoints/gru_model.pt")
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):.3f}")
    print(f"Valid Loss: {valid_loss:.3f} | Valid PPL: {np.exp(valid_loss):.3f}")


# EVALUATE

In [ ]:
# Evaluasi Model
model.load_state_dict(torch.load("gru_model.pt"))
test_loss = evaluate_fn(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):.3f}")

In [ ]:
# sentence = "Dia sangat pintar."
translated = translate_sentence(sentence, model, tokenizer, en_vocab, id_vocab, "<sos>", "<eos>", device)
print("Terjemahan:", translated)

Terjemahan: . . . . . . . . . . . . . . . . . . . . . . . .


In [ ]:
def translate_sentence(
    sentence,
    model,
    tokenizer,
    en_vocab,
    id_vocab,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
    max_repetition=3  # Batas maksimum pengulangan token tidak bermakna
):
    model.eval()
    
    # Tokenisasi input
    tokens = [sos_token] + tokenizer.tokenize(sentence.lower())[:1000] + [eos_token]
    numericalized = [id_vocab[token] if token in id_vocab else id_vocab["<unk>"] for token in tokens]
    sentence_tensor = torch.LongTensor(numericalized).unsqueeze(1).to(device)

    # Encoder
    with torch.no_grad():
        context = model.encoder(sentence_tensor)

    # Decoder
    trg_tokens = [en_vocab[sos_token]]
    hidden = context
    last_token = None
    repeat_count = 0

    for _ in range(max_output_length):
        trg_tensor = torch.LongTensor([trg_tokens[-1]]).to(device)
        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden, context)
        
        # Ambil token dengan probabilitas tertinggi
        top1 = output.argmax(1).item()
        trg_tokens.append(top1)

        # Cek pengulangan token tidak bermakna
        current_token = en_vocab.lookup_token(top1)
        if current_token in [".", "<pad>", "<unk>"]:  # Token yang dianggap tidak bermakna
            if current_token == last_token:
                repeat_count += 1
            else:
                repeat_count = 1
            if repeat_count >= max_repetition:
                break  # Hentikan jika pengulangan melebihi batas
        else:
            repeat_count = 0  # Reset jika token bermakna
        
        last_token = current_token

        # Hentikan jika menemukan <eos>
        if top1 == en_vocab[eos_token]:
            break

    # Konversi ke string
    translated_tokens = en_vocab.lookup_tokens(trg_tokens[1:-1])  # Hapus <sos> & <eos>
    translated_sentence = " ".join(translated_tokens)
    return translated_sentence

# Contoh penggunaan
sentence = "aku akan makan"
translated = translate_sentence(sentence, model, tokenizer, en_vocab, id_vocab, "<sos>", "<eos>", device)
print("Terjemahan:", translated)

Terjemahan: . .
